In [1]:
import numpy as np
import pandas as pd

In [2]:
from featurewiz import LazyTransformer, SuloRegressor, SuloClassifier
from featurewiz import FeatureWiz, cross_val_model_predictions
from featurewiz import print_regression_metrics, print_classification_metrics,print_sulo_accuracy

tensorflow>= 2.5 not installed in machine. Please install and try again. 
Imported lazytransform v1.10. 

Imported featurewiz 0.4.9. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True, scalers="std",
        		category_encoders="auto", add_missing=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [3]:
trainfile = 'c:/users/ram/documents/ram/data_sets/kaggle/diabetes.csv'
datapath = '../Ram/Data_Sets/'
filename = 'winequality.csv'
filename = 'affairs.csv'
trainfile = datapath+filename
sep = ','
dft = pd.read_csv(trainfile,sep=sep)
dft.drop(['affairs','affair'],axis=1, inplace=True)
print(dft.shape)
dft.head()

(6366, 9)


,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair_multiclass
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4


In [4]:
from featurewiz import LazyTransformer, get_class_distribution, get_class_weights

In [5]:
target = 'affair_multiclass'
#target = 'quality'
modeltype = 'Multi_Classification'
preds = [x for x in list(dft) if x not in target]
dft[target].nunique()

7

In [6]:
from sklearn.model_selection import train_test_split
from featurewiz import FE_kmeans_resampler
if modeltype == 'Regression':
    X_train, X_test, y_train, y_test = train_test_split(dft[preds], dft[target], test_size=0.20, random_state=1,)
    X_train_over, y_train_over = FE_kmeans_resampler(X_train, y_train, target, smote='',verbose=0)
    print(X_train_over.shape, X_test.shape)
    #train, test = pd.concat([X_train_over, pd.Series(y_train_over,name=target)], axis=1), pd.concat([X_test, y_test], axis=1)
    train, test = train_test_split(dft, test_size=0.20, random_state=42)
else:
    X_train, X_test, y_train, y_test = train_test_split(dft[preds], dft[target], test_size=0.20, 
                                                    stratify=dft[target],
                                                    random_state=42)
    train, test = train_test_split(dft, test_size=0.20, random_state=42,
                                                    stratify=dft[target]
                                                   )
print(X_train.shape, X_test.shape)

(5092, 8) (1274, 8)


In [7]:
dae_dicto = {
    'noise_factor': 0.2,
    'encoding_dim': 10,
    'epochs': 100, 
    'batch_size': 32,
    'simple_architecture': None
         }
vae_dicto = {
    'intermediate_dim': 32,
    'latent_dim': 4,
    'epochs': 100, 
    'batch_size': 32,
    'learning_rate': 0.001
         }
gan_dicto = {
    'gan_model':None,
    'input_dim': 100,
    'embedding_dim': 100, 
    'epochs': 100, 
    'num_synthetic_samples': 400,
         }

In [8]:
scaler = FeatureWiz(feature_engg = 'vae_add', nrows=None, transform_target=True, scalers="MinMax",
        		category_encoders="auto", add_missing=False, verbose=0, imbalanced=False,
                   ae_options=vae_dicto)

featurewiz is given 0.9 as correlation limit...
    final list of feature engineering given: ['vae_add']
    final list of category encoders given: ['onehot', 'label']
AutoEncoder VariationalAutoEncoder(batch_size=32, epochs=100, intermediate_dim=32)
    AE options: dict_items([('intermediate_dim', 32), ('latent_dim', 4), ('epochs', 100), ('batch_size', 32), ('learning_rate', 0.001)])
    final list of scalers given: [minmax]


In [9]:
# Load and preprocess your dataset
# Assuming X_train and y_train are your training data and labels
# Scale the data
#scaler = MinMaxScaler()
encoded_X_train, encoded_y_train = scaler.fit_transform(X_train, y_train)

Loaded input data. Shape = (5092, 8)
#### Starting featurewiz transform for train data ####
No groupby features created since no groupby feature engg specified
No interactions created for categorical vars since no interactions feature engg specified
    Single_Label Multi_Classification problem 
Shape of dataset: (5092, 8). Now we classify variables into different types...
Time taken to define data pipeline = 1 second(s)
No model input given...
Lazy Transformer Pipeline created...
    transformed target from object type to numeric
    Time taken to fit dataset = 1 second(s)
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (5092, 8)
Using Variational Auto Encoder to extract features...
Epoch 1/100
128/128 [==============================] - 1s 4ms/step - loss: 0.9101 - val_loss: 0.7354 - lr: 0.0010
Epoch 2/100
128/128 [==============================] - 0s 2ms/step - loss: 0.6526 - val_loss: 0.6324 - lr: 0.0010
Epoch 3/100
128/128 [======================

In [10]:
### Since you modified y_train to numeric, you must do same for y_test
encoded_X_test = scaler.transform(X_test)
y_test = scaler.lazy.yformer.transform(y_test)

#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (1274, 8)
    Beware! feature_engg will add 100s, if not 1000s of additional features to your dataset!
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (1274, 8)
Shape of transformed data due to auto encoder = (1274, 12)
Returning dataframe with 5 features 


In [11]:
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
# Train an SVM classifier on the encoded features
# svm_classifier = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', probability=True)
# Use the Random Forest Classifier with the corrected class weights which seems to work very well
### get_class_weights has lower ROC_AUC but get_class_distribution has higher F1 scores
class_weights_dict_corrected = get_class_distribution(encoded_y_train)
class_weights_dict_corrected = dict(zip(class_weights_dict_corrected.keys(),np.int8(np.reciprocal(list(class_weights_dict_corrected
                                                        .values())/max(list(class_weights_dict_corrected.values()))))))
svm_classifier = RandomForestClassifier(n_estimators=200, 
                                        #class_weight=class_weights_dict_corrected, 
                                        random_state=42)

svm_classifier.fit(encoded_X_train, encoded_y_train)

# Make predictions on the test set using the SVM classifier
svm_predictions = svm_classifier.predict(encoded_X_test)
svm_probas = svm_classifier.predict_proba(encoded_X_test)
# Calculate accuracy
accuracy = balanced_accuracy_score(y_test, svm_predictions)
if modeltype == 'Regression':
    print_regression_metrics(y_test, y_pred)
else:
    print_classification_metrics(y_test, svm_predictions, svm_probas, verbose=1)

Bal accu 15%
ROC AUC = 0.62
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      1050
           1       0.14      0.05      0.07        86
           2       0.15      0.04      0.07        48
           3       0.05      0.04      0.04        26
           4       0.00      0.00      0.00        35
           5       0.00      0.00      0.00        17
           6       0.00      0.00      0.00        12

    accuracy                           0.79      1274
   macro avg       0.17      0.15      0.15      1274
weighted avg       0.70      0.79      0.74      1274

final average balanced accuracy score = 0.15


In [12]:
class_weights_dict_corrected

{0: 1, 1: 12, 2: 21, 3: 40, 4: 30, 5: 58, 6: 58}

In [13]:
fwiz = FeatureWiz(corr_limit=0.90, feature_engg='', category_encoders='', transform_target=True,
                 dask_xgboost_flag=False, nrows=None, verbose=0) 
X_train_selected, y_train = fwiz.fit_transform(X_train, y_train) 

featurewiz is given 0.9 as correlation limit...
    Skipping feature engineering since no feature_engg input...
    final list of category encoders given: ['label', 'label']
    final list of scalers given: []
Loaded input data. Shape = (5092, 8)
#### Starting featurewiz transform for train data ####
    Single_Label Multi_Classification problem 
Shape of dataset: (5092, 8). Now we classify variables into different types...
Time taken to define data pipeline = 1 second(s)
No model input given...
Lazy Transformer Pipeline created...
    transformed target from object type to numeric
    Time taken to fit dataset = 1 second(s)
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (5092, 8)
    Single_Label Multi_Classification problem 
Starting SULOV with 8 features...
    Selecting all (8) variables since none of numeric vars are highly correlated...
Performing recursive XGBoost feature selection from 8 features...
    time taken to run entire featurewiz = 

In [14]:
X_test_selected = fwiz.transform(X_test) 

#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (1274, 8)
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (1274, 8)
Returning dataframe with 7 features 


In [15]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, classification_report
# Correctly computing class weights for the classes present in the training set
class_weights_dict_corrected = get_class_distribution(y_train)
class_weights_dict_corrected = dict(zip(class_weights_dict_corrected.keys(),np.int8(np.reciprocal(list(class_weights_dict_corrected
                                                        .values())/max(list(class_weights_dict_corrected.values()))))))

# Updating the Random Forest Classifier with the corrected class weights
rf_classifier = RandomForestClassifier(n_estimators=200, 
                                       #class_weight=class_weights_dict_corrected,
                                       random_state=42)

# Fitting the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluating the classifier
y_probas = rf_classifier.predict_proba(X_test)

if modeltype == 'Regression':
    print_regression_metrics(y_test, y_pred)
else:
    print_classification_metrics(y_test, y_pred, y_probas, verbose=1)

Bal accu 16%
ROC AUC = 0.69
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      1050
           1       0.14      0.05      0.07        86
           2       0.15      0.04      0.07        48
           3       0.00      0.00      0.00        26
           4       0.27      0.09      0.13        35
           5       0.00      0.00      0.00        17
           6       0.00      0.00      0.00        12

    accuracy                           0.79      1274
   macro avg       0.20      0.16      0.16      1274
weighted avg       0.71      0.79      0.74      1274

final average balanced accuracy score = 0.16


In [16]:
class_weights_dict_corrected = dict(zip(class_weights_dict_corrected.keys(),np.int8(np.reciprocal(list(class_weights_dict_corrected
                                                        .values())/max(list(class_weights_dict_corrected.values()))))))
class_weights_dict_corrected

{0: 58, 1: 4, 2: 2, 3: 1, 4: 1, 5: 1, 6: 1}